<a href="https://colab.research.google.com/github/Alimiji/Ali_siteweb/blob/main/Copie_de_Partie2_ALI_ADJAFATOU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Application des équations de Bellman sur un projet réel**

##**1. Téléchargement du programme Python**

In [ ]:
import os
import sys

sys.path.insert(1, "/content/ApplicationRL")

In [ ]:
!pip -q install meshio

##**2. Découverte de l'environnement de travail**

In [ ]:
from magasin import Magasin
from utils import bgr8_to_jpeg
import ipywidgets
import numpy as np

from IPython.display import display

#####**2.1 Instanciation de la classe Magasin**

In [ ]:
Magasin = Magasin("maillage_carte.msh","image_magasin.png")

In [ ]:
Magasin.RECOMPENSE = 1
Magasin.RECOMPENSE_MUR = -10
Magasin.RECOMPENSE_NON_CIBLE = 0
Magasin.PROBA_MAX = 0.98

Magasin.RAZ()

Initialisation de la carte ...

Construction de la table des transitions ...
Initialisation de la table des valeurs des actions...
Initialisation de la table des valeurs des états...


#####**2.2 Saisie de l'objectif**

In [ ]:
def AfficheSelectionImage(change):
    numero = change['new']
    Magasin.ETAT_CIBLE = numero
    widget_image.value = bgr8_to_jpeg(Magasin.AfficheObjectifSurImage())

In [ ]:
# Création des widgets
widget_image = ipywidgets.Image(format='png',value=bgr8_to_jpeg(Magasin.image_magasin),width=640)
slider_numero = ipywidgets.IntSlider(value=80,min=0,max=Magasin.nombre_etats-1)

# Création du lien avec le slider
slider_numero.observe(AfficheSelectionImage, names='value')

# Création de l'interface d'acquisition
widget_presentation = ipywidgets.VBox([widget_image, slider_numero])

display(widget_presentation)

In [ ]:
# Affiche la table de transition
P = Magasin.table_transitions

# Affichage des valeurs contenues dans la table

print(P)


[[[(225, 0.98,   0.) (227, 0.01,   0.) (226, 0.01,   0.)]
  [(225, 0.01,   0.) (227, 0.98,   0.) (226, 0.01,   0.)]
  [(225, 0.01,   0.) (227, 0.01,   0.) (226, 0.98,   0.)]]

 [[(263, 0.98,   0.) (259, 0.01,   0.) (  3, 0.01,   0.)]
  [(263, 0.01,   0.) (259, 0.98,   0.) (  3, 0.01,   0.)]
  [(263, 0.01,   0.) (259, 0.01,   0.) (  3, 0.98,   0.)]]

 [[(222, 0.98,   0.) (267, 0.01,   0.) (299, 0.01,   0.)]
  [(222, 0.01,   0.) (267, 0.98,   0.) (299, 0.01,   0.)]
  [(222, 0.01,   0.) (267, 0.01,   0.) (299, 0.98,   0.)]]

 ...

 [[(  2, 0.98,   0.) (  6, 0.01,   0.) (299, 0.01, -10.)]
  [(  2, 0.01,   0.) (  6, 0.98,   0.) (299, 0.01, -10.)]
  [(  2, 0.01,   0.) (  6, 0.01,   0.) (299, 0.98, -10.)]]

 [[(249, 0.98,   0.) (251, 0.01,   0.) (300, 0.01, -10.)]
  [(249, 0.01,   0.) (251, 0.98,   0.) (300, 0.01, -10.)]
  [(249, 0.01,   0.) (251, 0.01,   0.) (300, 0.98, -10.)]]

 [[(289, 0.98,   0.) (286, 0.01,   0.) (301, 0.01, -10.)]
  [(289, 0.01,   0.) (286, 0.98,   0.) (301, 0.01, -10.)

In [ ]:
print(P.shape)

(302, 3, 3)


**Itération état des valeurs**

In [ ]:
#On reinitialise l'environnement

Magasin.RECOMPENSE = 50
Magasin.RECOMPENSE_MUR = -0.1
Magasin.RECOMPENSE_NON_CIBLE = -0.1
Magasin.PROBA_MAX = 0.9

Magasin.RAZ()

Initialisation de la carte ...

Construction de la table des transitions ...
Initialisation de la table des valeurs des actions...
Initialisation de la table des valeurs des états...


In [ ]:
# Algorithme d'itération des valeurs 
gamma = 0.9     # Facteur de remise
theta = 1e-10   # Seuil de précision

# Initialisation de la fonction des valeurs d'états à 0
Magasin.V_table = np.zeros(Magasin.nombre_etats, dtype=np.float32)
V_ = np.zeros(Magasin.nombre_etats, dtype=np.float32)


# Répétition de l'algorithme jusqu'à convergence
delta = 2*theta

while delta > theta:
  # Pour chaque état de l'environnement
  for etat in range(Magasin.nombre_etats):
    # Initialise la fonction de valeurs d'états pour chaque action à 0
    V = np.zeros((3), dtype=np.float32)

    # Calcule la nouvelle valeur de l'état en cours à l'aide
    # de l'équation optimale des valeurs d'états de Bellman
    for action in range(3):
      # Calcul la valeur de l'état en cours pour l'action courante
      # ainsi que les valeurs d'actions
      for etat_suivant in range(3):
        proba = Magasin.table_transitions[etat,action,etat_suivant]['proba_transition']
        recompense = Magasin.table_transitions[etat,action,etat_suivant]['recompense']
        index_etat_suivant = Magasin.table_transitions[etat,action,etat_suivant]['index_etat_suivant']

        # Itération de la valeur de l'état courant pour l'action en cours
        V[action] += proba*(recompense + gamma*Magasin.V_table[index_etat_suivant])

        # Itération de la valeur des actions 
        Magasin.Q_table[etat,action] +=  proba*(recompense + gamma*Magasin.V_table[index_etat_suivant])

    # Enregistre la valeur d'état maximale parmi toutes les actions
    Magasin.V_table[etat] = np.amax(V)
    
  # Compare la fonction des valeurs des états obtenue avec la précédente
  delta = np.max(np.abs(V_ - Magasin.V_table))
  print(delta)

  # Sauvegarde la fonction des valeurs des états pour la prochaine itération
  V_ = Magasin.V_table.copy()

0.19464503
0.16737646
0.14335655
0.12060344
0.105692714
0.06344411
0.05651492
0.050293744
0.04474777
0.039819658
0.03567463
0.032021046
0.028720975
0.025744319
0.023062646
0.020649433
0.018485308
0.016583383
0.014876485
0.013342977
0.011965692
0.010729015
0.009618819
0.008622408
0.0077282786
0.0069261193
0.00620836
0.0055652857
0.00498873
0.0044716597
0.0040080547
0.003592372
0.0032197833
0.0028856397
0.002586186
0.0023177862
0.0020771027
0.001861453
0.0016680956
0.0014948249
0.0013395548
0.0012003779
0.0010756254
0.0009638667
0.0008636713
0.0007738471
0.00069344044
0.0006213784
0.0005567074
0.0004988313
0.00044697523
0.0004004836
0.00035881996
0.00032150745
0.00028806925
0.0002580881
0.00023126602
0.00020712614
0.00018560886
0.00016629696
0.00014895201
0.0001335144
0.00011962652
0.00010710955
9.596348e-05
8.59499e-05
7.70092e-05
6.902218e-05
6.186962e-05
5.5372715e-05
4.965067e-05
4.4465065e-05
3.9815903e-05
3.5703182e-05
3.194809e-05
2.861023e-05
2.5689602e-05
2.2947788e-05
2.0623207

In [ ]:
Magasin.Getdf_Vtable()

,Etat courant,Valeur
0,0,-0.999999
1,1,-0.999999
2,2,-0.999999
3,3,-0.999999
4,4,-0.999999
...,...,...
297,297,-0.999999
298,298,-0.999999
299,299,-0.999999
300,300,-0.999999


In [ ]:
# Affichage carte des valeurs d'etat
# Création de la colorMap
Magasin.CreationColorMap()

# Création des widgets
widget_image = ipywidgets.Image(format='png',value=bgr8_to_jpeg(Magasin.image_Vtable),width=640)

display(widget_image)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

In [ ]:
#Affichage du parcours
Magasin.Getdf_Qtable()

,Etat courant,Action,Valeur
0,0,0,-124.406548
1,0,1,-124.453964
2,0,2,-124.589691
3,1,0,-124.673172
4,1,1,-125.173943
...,...,...,...
901,300,1,-125.160194
902,300,2,-124.657730
903,301,0,-125.328461
904,301,1,-125.339500


In [ ]:
# Création des widgets
widget_image = ipywidgets.Image(format='png',value=bgr8_to_jpeg(Magasin.image_magasin_objectif),width=640)
widget_iteration = ipywidgets.Text(value="0",description="Itération: ")
widget_etat = ipywidgets.Text(value="0",description="État: ")

widget_presentation = ipywidgets.VBox([widget_iteration,widget_etat,widget_image])

In [28]:
import time

objectif_atteint = False
max_iteration = 500
freq_mise_a_jour = 1

Magasin.InitImageTrajectoire()

iteration = 0
etat_courant = Magasin._ETAT_DEPART

display(widget_presentation)

time.sleep(1)

while objectif_atteint is False and iteration <= max_iteration:
    widget_etat.value = str(etat_courant)

    # Récupère la table des valeurs d'action de l'état en cours
    Q_ = Magasin.Q_table[etat_courant,:]

    # Récupère l'action optimale
    # si plusieurs actions sont optimales, alors tire une au hasard
    #action = np.random.choice(np.where(Q_ == np.max(Q_))[0])
    action = np.argmax(Q_)

    # Simule l'action
    etat_courant, image_ = Magasin.SimuleAction(etat_courant,action)

    if etat_courant == Magasin.ETAT_CIBLE:
        objectif_atteint = True

    # Affiche la trajectoire
    if iteration%freq_mise_a_jour == 0:
      widget_image.value = bgr8_to_jpeg(image_)
      widget_iteration.value = str(iteration+1)

    iteration = iteration + 1
    time.sleep(0.7)

**Avec l'algorithme d'itérations des valeurs nous pouvons améliorer / optimiser le chemin vers la cible en modifiant les récompenses, nous avons essayé de donner une plus grande récompense au niveau de la cible et procéder a plusieurs essais pour avoir un chemin plus ou moins optimal.
Dans cet exercice nous n'avons pas trouvé les bons parametres, car l'agent n'arretait pas d'osciller entre 4 premiers triangles**

**Evaluation des stratégies**

In [29]:
#On reinitialise l'environnement

Magasin.RECOMPENSE = 50
Magasin.RECOMPENSE_MUR = -0.1
Magasin.RECOMPENSE_NON_CIBLE = -0.1
Magasin.PROBA_MAX = 0.9

Magasin.RAZ()

Initialisation de la carte ...

Construction de la table des transitions ...
Initialisation de la table des valeurs des actions...
Initialisation de la table des valeurs des états...


In [30]:
# Fonction permettant d'évaluer une stratégie
# pi :  pi(a|s)
# table_transition :   Table des transitions
# V_ :  Fonction des valeurs d'états issue de l'amélioration de la stratégie précédente
# gamma : Facteur de remise
# theta : Seuil déterminant la précision de l'évaluation

def evaluation_strategie(pi, table_transition, V_, gamma=0.99, theta=1e-10):
    # Répéter tant que l'algorithme n'a pas convergé
    delta = 2*theta
    while delta > theta:
        # Initialise la fonction valeur d'état à 0
        V = np.zeros(len(table_transition), dtype=np.float64)

        # Pour chaque état de l'environnement
        for s in range(len(table_transition)):
            # Calcule la nouvelle valeur de l'état en cours à l'aide de la
            # fonction des valeurs d'états issus de l'amélioration de la
            # stratégie précédente
            #for proba, etat_suivant, recompense, etat_terminal in table_transition[s][pi(s)]:
                proba = table_transition[etat,action,etat_suivant]['proba_transition']
                recompense = table_transition[etat,action,etat_suivant]['recompense']
                index_etat_suivant = table_transition[etat,action,etat_suivant]['index_etat_suivant']

                V[s] += proba*(recompense + gamma*V_[index_etat_suivant])
            
        # Compare la fonction des valeurs des états obtenue avec la précédente
        delta = np.max(np.abs(V_ - V))
        print(delta)

        # Sauvegarde la fonction des valeurs des états pour la prochaine itération
        V_ = V.copy()
    return V

In [31]:
# Fonction permettant d'améliorer une stratégie
#  pi :     Stratégie à améliorer
#  V :      Fonction de valeurs des états
#  table_transition :      Table de transition
#  gamma :  Facteur de remise

def amelioration_strategie(pi, V, table_transition, gamma=0.99):
    # Initalise la fonction des valeurs d'actions
    Q = np.zeros((len(table_transition), len(table_transition[0])), dtype=np.float64)

    # Initialise l'indicateur de strategie stabilisée
    strategie_stable = True

    # Pour chaque état de l'environnement
    for s in range(len(table_transition)):
        # Pour chaque action disponnible sur cet état
        for a in range(len(table_transition[s])):
            proba = table_transition[etat,action,etat_suivant]['proba_transition']
            recompense = table_transition[etat,action,etat_suivant]['recompense']
            index_etat_suivant = table_transition[etat,action,etat_suivant]['index_etat_suivant']
            # Calcul de la valeur d'action Q(s,a)
            #for proba, etat_suivant, recompense, etat_terminal in table_transition[s][a]:
            Q[s][a] += proba*(recompense + gamma*V[index_etat_suivant])
            
        # Récupère l'action la plus forte sur cet état
        a_max = np.argmax(Q[s,:])

        # Compare cette action avec celle donnée par la stratégie à optimiser
        if a_max != pi(s):
          strategie_stable = False

    #calculer la nouvelle politique qui va être comparée. L'itération s'arrêtera si la différence est inférieure au seuil
    nouvelle_pi = lambda s: {s:a for s, a in enumerate(np.argmax(Q, axis=1))}[s]
    return nouvelle_pi, strategie_stable

In [32]:
# Algorithme d'itération des stratégies 
gamma = 0.99    # Facteur de remise
theta = 1e-10   # Seuil de précision de l'évaluation des stratégies

# Initialise une stratégie aléatoire pi(s)
actions_aleatoires = np.random.choice([0,1,2],len(P))
pi = lambda s: {s:a for s, a in enumerate(actions_aleatoires)}[s]               # pi(s) = action de la stratégie pi

strategie_stable = False

# Initialisation de la fonction des valeurs d'états à 0
V_ = np.zeros(len(P), dtype=np.float64)

while strategie_stable is False:
  print('=================================================================')
  print ("Stratégie : %s" %{s:pi(s) for s in range(len(P))})
  print('=================================================================')
  # Évaluation de la stratégie
  V = evaluation_strategie(pi, P, V_, gamma, theta)

  # Amélioration de la stratégie
  pi, strategie_stable = amelioration_strategie(pi, V, P, gamma)

Stratégie : {0: 2, 1: 1, 2: 0, 3: 1, 4: 1, 5: 0, 6: 2, 7: 1, 8: 1, 9: 0, 10: 2, 11: 2, 12: 0, 13: 2, 14: 1, 15: 1, 16: 1, 17: 2, 18: 1, 19: 1, 20: 2, 21: 2, 22: 2, 23: 1, 24: 0, 25: 1, 26: 0, 27: 2, 28: 0, 29: 2, 30: 1, 31: 1, 32: 0, 33: 1, 34: 1, 35: 0, 36: 0, 37: 2, 38: 0, 39: 1, 40: 2, 41: 1, 42: 1, 43: 2, 44: 2, 45: 0, 46: 1, 47: 0, 48: 0, 49: 2, 50: 2, 51: 0, 52: 1, 53: 0, 54: 0, 55: 2, 56: 1, 57: 2, 58: 2, 59: 2, 60: 1, 61: 0, 62: 1, 63: 2, 64: 0, 65: 2, 66: 2, 67: 2, 68: 1, 69: 0, 70: 0, 71: 1, 72: 0, 73: 2, 74: 2, 75: 0, 76: 1, 77: 0, 78: 2, 79: 0, 80: 1, 81: 1, 82: 0, 83: 0, 84: 1, 85: 1, 86: 2, 87: 1, 88: 0, 89: 1, 90: 2, 91: 0, 92: 1, 93: 2, 94: 2, 95: 2, 96: 2, 97: 0, 98: 2, 99: 2, 100: 0, 101: 0, 102: 2, 103: 1, 104: 1, 105: 1, 106: 0, 107: 1, 108: 0, 109: 0, 110: 0, 111: 2, 112: 0, 113: 0, 114: 1, 115: 1, 116: 2, 117: 0, 118: 1, 119: 2, 120: 1, 121: 2, 122: 2, 123: 2, 124: 1, 125: 0, 126: 2, 127: 0, 128: 1, 129: 2, 130: 0, 131: 1, 132: 2, 133: 2, 134: 2, 135: 2, 136: 2, 1

**Stratégie optimale selon l'algorithme d'iteration des stratégies, nous n'avons pas pu dessiner la trajectoire avec cette grille, ce sera un point a travailler de facon personnelle : {0: 1, 1: 0, 2: 1, 3: 0, 4: 2, 5: 0, 6: 0, 7: 2, 8: 1, 9: 2, 10: 1, 11: 2, 12: 0, 13: 1, 14: 0, 15: 1, 16: 1, 17: 0, 18: 0, 19: 1, 20: 2, 21: 1, 22: 0, 23: 0, 24: 0, 25: 0, 26: 1, 27: 1, 28: 0, 29: 2, 30: 1, 31: 2, 32: 0, 33: 0, 34: 0, 35: 2, 36: 0, 37: 1, 38: 0, 39: 1, 40: 0, 41: 1, 42: 0, 43: 1, 44: 2, 45: 2, 46: 1, 47: 2, 48: 0, 49: 1, 50: 2, 51: 0, 52: 1, 53: 2, 54: 0, 55: 0, 56: 1, 57: 0, 58: 2, 59: 2, 60: 2, 61: 0, 62: 0, 63: 2, 64: 0, 65: 1, 66: 1, 67: 1, 68: 1, 69: 1, 70: 1, 71: 0, 72: 2, 73: 1, 74: 1, 75: 1, 76: 2, 77: 2, 78: 2, 79: 2, 80: 0, 81: 0, 82: 1, 83: 1, 84: 2, 85: 0, 86: 0, 87: 0, 88: 0, 89: 1, 90: 0, 91: 2, 92: 0, 93: 0, 94: 2, 95: 0, 96: 0, 97: 2, 98: 2, 99: 0, 100: 2, 101: 0, 102: 1, 103: 1, 104: 0, 105: 0, 106: 1, 107: 2, 108: 2, 109: 0, 110: 0, 111: 1, 112: 1, 113: 0, 114: 1, 115: 2, 116: 1, 117: 1, 118: 0, 119: 0, 120: 1, 121: 0, 122: 1, 123: 2, 124: 0, 125: 1, 126: 2, 127: 0, 128: 1, 129: 2, 130: 2, 131: 1, 132: 0, 133: 2, 134: 2, 135: 2, 136: 2, 137: 0, 138: 0, 139: 0, 140: 0, 141: 2, 142: 1, 143: 2, 144: 2, 145: 0, 146: 0, 147: 1, 148: 1, 149: 0, 150: 2, 151: 0, 152: 1, 153: 2, 154: 2, 155: 1, 156: 2, 157: 2, 158: 2, 159: 1, 160: 0, 161: 1, 162: 1, 163: 1, 164: 2, 165: 2, 166: 1, 167: 1, 168: 1, 169: 1, 170: 0, 171: 0, 172: 0, 173: 1, 174: 1, 175: 0, 176: 2, 177: 1, 178: 0, 179: 0, 180: 1, 181: 2, 182: 2, 183: 2, 184: 2, 185: 1, 186: 0, 187: 0, 188: 0, 189: 0, 190: 2, 191: 2, 192: 2, 193: 0, 194: 1, 195: 0, 196: 1, 197: 0, 198: 0, 199: 1, 200: 0, 201: 0, 202: 0, 203: 2, 204: 0, 205: 0, 206: 0, 207: 1, 208: 2, 209: 2, 210: 1, 211: 2, 212: 1, 213: 0, 214: 1, 215: 0, 216: 1, 217: 1, 218: 2, 219: 0, 220: 0, 221: 1, 222: 2, 223: 1, 224: 2, 225: 1, 226: 0, 227: 0, 228: 0, 229: 0, 230: 0, 231: 1, 232: 2, 233: 0, 234: 1, 235: 2, 236: 1, 237: 0, 238: 1, 239: 2, 240: 1, 241: 2, 242: 2, 243: 1, 244: 1, 245: 0, 246: 1, 247: 1, 248: 2, 249: 1, 250: 2, 251: 1, 252: 2, 253: 1, 254: 2, 255: 2, 256: 0, 257: 0, 258: 1, 259: 1, 260: 1, 261: 2, 262: 1, 263: 2, 264: 1, 265: 0, 266: 2, 267: 0, 268: 0, 269: 0, 270: 1, 271: 2, 272: 2, 273: 0, 274: 1, 275: 0, 276: 1, 277: 2, 278: 2, 279: 0, 280: 2, 281: 2, 282: 0, 283: 0, 284: 2, 285: 2, 286: 2, 287: 1, 288: 1, 289: 0, 290: 2, 291: 0, 292: 0, 293: 0, 294: 1, 295: 0, 296: 2, 297: 2, 298: 1, 299: 2, 300: 2, 301: 0}**
